In [ ]:
# 1. No, this can correct overfitting in boostiong models 

# 2. Yes

# 3. A decision tree only conciders one tree whereas a random forest uses many trees. 
# The random forest uses the bagging approach to decide which tree to use and is therefore much more accurate than a single tree. 

# 4. c

# 5. See attatched document

In [11]:
import boto3
import pandas as pd
import numpy as np
import statistics as st

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'ryan-greiner-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading CSV file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


# Pre-Processing

In [38]:
X = heart.drop(columns = 'TenYearCHD', axis = 1)
Y = heart['TenYearCHD']

importance = list()
rf1_recall = list()
rf2_recall = list()
rf3_recall = list()

# Random Forest Importances

In [34]:
for i in range(0, 100):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)
    rf_md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)
    importance.append(rf_md.feature_importances_)

importances = pd.DataFrame({'Feature':X_train.columns, 'Average Importance': np.mean(importance, axis = 0)})
importances = importances.sort_values(by = 'Average Importance', ascending = False)
importances

,Feature,Average Importance
10,sysBP,0.135151
12,BMI,0.127262
1,age,0.124559
9,totChol,0.122128
14,glucose,0.119763
11,diaBP,0.119078
13,heartRate,0.095815
4,cigsPerDay,0.050357
2,education,0.036807
0,male,0.021248


In [40]:
# The five most important variables are sysBP, BMI, age, totChol, and glucose
X = heart[['sysBP', 'BMI', 'age', 'totChol', 'glucose']]

# Random Forest Model

In [45]:
for i in range(0, 100):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)
    rf1_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    rf1_pred = rf1_md.predict_proba(X_test)[:, 1]
    rf1_label = np.where(rf1_pred < 0.1, 0, 1)
    rf1_recall.append(recall_score(Y_test, rf1_label))
    
    rf2_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)
    rf2_pred = rf2_md.predict_proba(X_test)[:, 1]
    rf2_label = np.where(rf2_pred < 0.1, 0, 1)
    rf2_recall.append(recall_score(Y_test, rf2_label))
    
    rf3_md = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)
    rf3_pred = rf3_md.predict_proba(X_test)[:, 1]
    rf3_label = np.where(rf3_pred < 0.1, 0, 1)
    rf3_recall.append(recall_score(Y_test, rf3_label))

pd.DataFrame({'Max Depth': [3, 5, 7], 'Average Recall Score':[np.mean(rf1_recall),
                                                              np.mean(rf2_recall),
                                                              np.mean(rf3_recall)]})

,Max Depth,Average Recall Score
0,3,0.839529
1,5,0.824188
2,7,0.809740


In [ ]:
# The best model to predict TenYearCHD is the random forest model with max depth of 3 